## Convolutional Neural network

인간의 시신경 모방, 이미지의 공간 정보를 보존하는 형태의 신경망

#### Convolution Layer

- 합성곱 연산 $(f*g)(x)=\int_{-\infty}^{\infty} f(t)g(x-t)dt $을 적용한 layer
- 어떤 이미지에 대해, 해당 이미지의 각 부분과 필터(커널, 윈도우라고도 함)와 합성곱 연산 수행
- 이때 이미지를 input, 필터를 layer의 weight으로 생각
- 이미지 크기가 아닌, 필터의 크기만큼의 weight만 사용하므로 학습할 weight 수 DNN에 비해 적음
- Convolution Layer의 각 node는 다음 layer의 일부 node와만 연결됨 (fully connected가 아닌 localy connected)
- Stride: 필터가 한 번에 움직이는 픽셀 수
- Padding: 이미지 끝 부분의 정보 손실, 혹은 이미지 크기 감소를 막기 위해 이미지 주변을 0으로 채우는 방법

#### Parameter, Feature Map size in Convolution Layer
1. 주어진 값
- Input Image 차원: $W\times H\times D$
- Filter 개수: $N$, Filter 크기: $F\times F$
- Stride: $S$, Padding: $P$
2. 계산되는 값
- Feature Map 차원: $W'\times H' \times D'$
- $W + 2P = S*(W'-1)+F \rightarrow W'=(W+2P-F)/S+1$
- 같은 원리로 $H' = (W+2P-F)/S+1$
- $D' = N$
- 파라미터 수 $N\times (F^2\times D+D (\text{bias}))$

#### Pooling Layer
- input의 공간적 정보를 유지하며 input의 크기를 줄여주는 layer
- Convolution Layer과 유사하게 필터가 움직이며 이미지 각 부분의 최댓값(max pooling)이나 평균(average pooling) 반환
- 주로 max pooling 사용

#### Parameter, Feature Map size in Pooling Layer
1. 주어진 값
- Input Image 차원: $W\times H\times D$
- Filter 크기: $F\times F$, Stride: $S$, Padding:$P$
2. 계산되는 값
- Feature Map 차원: $W'\times H'\times D'$
- $W' = (W+2P-F)/S+1, H' = (H+2P-F)/S+1$
- $D' = D$
- 파라미터 X

#### MNIST CNN

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class MnistCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) #input channel, output channel(filter 개수) 순
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(14*14*64, 128) 
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x): # x = (28, 28, 1)
        x = self.conv1(x) # x = (28, 28, 32)
        x = self.relu(x)
        x = self.conv2(x) # x = (28, 28, 64)
        x = self.relu(x)
        x = self.pool(x) # x = (14, 14, 64)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [4]:
model = MnistCNN()

learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
for t in range(epochs):
    print(f"Epoch {t+1}")
    #train
    model.train()
    for X, y in train_dataloader:
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    #test
    model.eval()
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct}%, Avg loss: {test_loss} \n")

Epoch 1
Test Error: 
 Accuracy: 88.02%, Avg loss: 0.32871700851780594 

Epoch 2
Test Error: 
 Accuracy: 88.97%, Avg loss: 0.2998134306851466 

Epoch 3
Test Error: 
 Accuracy: 90.5%, Avg loss: 0.2697259651817334 

Epoch 4
Test Error: 
 Accuracy: 90.95%, Avg loss: 0.2662014054122624 

Epoch 5
Test Error: 
 Accuracy: 91.14%, Avg loss: 0.3102726412901453 

